In [ ]:
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline


files = glob.glob("/content/drive/MyDrive/apartment_price_2024_fall/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0, low_memory=False, encoding="utf-8"))
df = pd.concat(data_list)






In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847189 entries, 14144164 to 34012550
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   種類            847189 non-null  object 
 1   地域            0 non-null       float64
 2   市区町村コード       847189 non-null  int64  
 3   都道府県名         847189 non-null  object 
 4   市区町村名         847189 non-null  object 
 5   地区名           846528 non-null  object 
 6   最寄駅：名称        844312 non-null  object 
 7   最寄駅：距離（分）     823886 non-null  object 
 8   間取り           813630 non-null  object 
 9   面積（㎡）         847189 non-null  object 
 10  土地の形状         0 non-null       float64
 11  間口            0 non-null       float64
 12  延床面積（㎡）       0 non-null       float64
 13  建築年           822326 non-null  object 
 14  建物の構造         821437 non-null  object 
 15  用途            745423 non-null  object 
 16  今後の利用目的       476687 non-null  object 
 17  前面道路：方位       0 non-null       float64
 18  

In [ ]:
import re

def data_pre(df):
    # すべてがNULL値の列を削除
    nonnull_list = [col for col in df.columns if df[col].count() == 0]
    df = df.drop(nonnull_list, axis=1)

    # 不要な列の削除
    if "市区町村名" in df.columns:
        df = df.drop("市区町村名", axis=1)
    if "種類" in df.columns:
        df = df.drop("種類", axis=1)

    # 距離を数値に置換
    dis = {
        "30分～60分": 45,
        "1H～1H30": 75,
        "2H～": 120,
        "1H30～2H": 105,
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    # 面積を数値に置換
    df["面積（㎡）"] = df["面積（㎡）"].replace("2,000㎡以上", 2000).astype(float)

   # 建築年の変換 with improved handling for different formats
    def convert_year(value):
        if isinstance(value, float) or pd.isna(value):
            return np.nan  # Handle missing values

        # More flexible regex for year extraction
        match = re.search(r"(\d{2,4})", str(value))  # Match 2-4 digits
        if match:
            year_str = match.group(1)

            # More flexible era matching using 'contains'
            if "平成" in str(value):
                year = 36 - float(year_str)
            elif "令和" in str(value):
                year = 4 - float(year_str)
            elif "昭和" in str(value):
                year = 96 - float(year_str)
            elif str(value) == "戦前":
                year = 76
            else:
                try:
                    year = float(year_str)
                except ValueError:
                    year = np.nan

            return year
        else:
            return np.nan

    df["建築年"] = df["建築年"].apply(convert_year)

    # 四半期を数値に置換する関数
    def convert_quarter_to_float(value):
        # Modified to handle variations in the quarter representation
        # Check if value is already a float, if so, return it directly
        if isinstance(value, float):
            return value

        year = {
            "年第1四半期": ".25",
            "年第2四半期": ".50",
            "年第3四半期": ".75",
            "年第4四半期": ".99"  # Corrected the key to "年第4四半期"
        }
        for k, v in year.items():
            if k in value:
                year_str = value.split('年')[0]  # Extract the year part
                return float(year_str + v)  # Combine year and quarter value
        # If no matching key is found, raise a ValueError with the problematic value
        raise ValueError(f"変換できない値: {value}")

    # 取引時点列を変換
    df["取引時点"] = df["取引時点"].apply(convert_quarter_to_float)


    # 特定の列をカテゴリ型に変換
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り",
                "建物の構造", "用途", "今後の利用目的", "都市計画",
                "改装", "取引の事情等"]:
        df[col] = df[col].astype("category")

    return df


In [ ]:
df = data_pre(df)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 847189 entries, 14144164 to 34012550
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       847189 non-null  int64   
 1   都道府県名         847189 non-null  category
 2   地区名           846528 non-null  category
 3   最寄駅：名称        844312 non-null  category
 4   最寄駅：距離（分）     823886 non-null  float64 
 5   間取り           813630 non-null  category
 6   面積（㎡）         847189 non-null  float64 
 7   建築年           821724 non-null  float64 
 8   建物の構造         821437 non-null  category
 9   用途            745423 non-null  category
 10  今後の利用目的       476687 non-null  category
 11  都市計画          825452 non-null  category
 12  建ぺい率（％）       820465 non-null  float64 
 13  容積率（％）        820465 non-null  float64 
 14  取引時点          847189 non-null  float64 
 15  改装            759690 non-null  category
 16  取引の事情等        19853 non-null   category
 17  取引価格（総額）_log  847189 non-

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,
14144164,14116,神奈川県,中田南,踊場,4.0,１Ｋ,15.0,1991.0,ＲＣ,住宅,住宅,準住居,60.0,200.0,2017.50,未改装,NaN,6.799341
14045260,14105,神奈川県,井土ケ谷中町,井土ケ谷,3.0,３ＬＤＫ,70.0,2003.0,ＲＣ,住宅,住宅,準工業,60.0,200.0,2017.25,未改装,NaN,7.591065
14208249,14136,神奈川県,平,宮前平,28.0,２ＬＤＫ,65.0,1976.0,ＲＣ,住宅,住宅,２中住専,60.0,200.0,2018.75,NaN,NaN,6.698970
14002281,14101,神奈川県,豊岡町,鶴見,4.0,１ＬＤＫ,40.0,2003.0,ＲＣ,住宅,住宅,商業,80.0,400.0,2021.50,未改装,NaN,7.278754
14209402,14136,神奈川県,鷺沼,鷺沼,12.0,１Ｋ,15.0,1991.0,ＲＣ,NaN,住宅,１低住専,50.0,80.0,2017.75,未改装,NaN,6.919078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34039913,34202,広島県,三条,呉,10.0,３ＬＤＫ,65.0,1996.0,ＳＲＣ,住宅,住宅,１種住居,60.0,300.0,2021.50,未改装,NaN,7.079181
34015299,34104,広島県,天満町,広島,45.0,４ＬＤＫ,80.0,1994.0,ＳＲＣ,住宅,住宅,商業,80.0,400.0,2015.75,未改装,NaN,7.301030
34007436,34102,広島県,光町,広島,9.0,１Ｋ,25.0,2008.0,ＲＣ,住宅,NaN,近隣商業,80.0,400.0,2008.50,未改装,NaN,7.079181
